In [29]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random

# Card and Deck classes
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10,
          'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        self.image = ImageTk.PhotoImage(Image.open(f"images/{rank}_of_{suit}.png").resize((80, 120)))

    def __str__(self):
        return f"{self.rank} of {self.suit} ({self.value})"

class Deck:
    def __init__(self):
        self.all_cards = [Card(suit, rank) for suit in suits for rank in ranks]

    def shuffle(self):
        random.shuffle(self.all_cards)

    def deal_one(self):
        return self.all_cards.pop()

class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0

    def add_card(self, card):
        self.cards.append(card)
        self.value += card.value
        if card.rank == 'Ace':
            self.aces += 1
        self.adjust_for_ace()

    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

    def __str__(self):
        return ', '.join(str(card) for card in self.cards)

class Chips:
    def __init__(self, total=100):
        self.total = total
        self.bet = 0

    def win_bet(self):
        self.total += self.bet

    def lose_bet(self):
        self.total -= self.bet

class BlackjackGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Blackjack")
        self.root.state('zoomed')  # Start in maximized mode (not fullscreen)
        self.root.configure(bg="green")

        self.deck = Deck()
        self.deck.shuffle()
        self.player_hand = Hand()
        self.dealer_hand = Hand()
        self.chips = Chips()
        self.player_turn = True

        # Main frame (always visible, even in restore down mode)
        self.main_frame = tk.Frame(root, bg="dark green")
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Left frame (fixed width for controls)
        self.left_frame = tk.Frame(self.main_frame, bg="dark green", width=300)
        self.left_frame.grid(row=0, column=0, sticky="ns", padx=10, pady=10)

        # Right frame (expands for labels & canvases)
        self.right_frame = tk.Frame(self.main_frame, bg="dark green")
        self.right_frame.grid(row=0, column=1, sticky="nsew", padx=10, pady=10)

        self.main_frame.grid_columnconfigure(1, weight=1)  # Right frame expands
        self.main_frame.grid_rowconfigure(0, weight=1)  # Allow expansion

        self.rules_window = None  # To track if the Rules window is open

        # Rules Button (Top Right Corner)
        self.rules_button = tk.Button(root, text="Rules", font=("Helvetica", 12, "bold"),
                                      width=10, bg="#FFD700", fg="black",
                                      command=self.show_rules)
        self.rules_button.place(relx=1, x=-10, y=10, anchor="ne")  # Top-right positioning

        

        # Labels (Player's and Dealer's text is centered above the canvas)
        self.player_label = tk.Label(self.right_frame, text="Player's Hand: []", font=("Helvetica", 14), bg="dark green", fg="white")
        self.player_label.pack(anchor=tk.CENTER, pady=5)

        self.dealer_label = tk.Label(self.right_frame, text="Dealer's Hand: []", font=("Helvetica", 14), bg="dark green", fg="white")
        self.dealer_label.pack(anchor=tk.CENTER, pady=5)

        self.score_label = tk.Label(self.left_frame, text="Chips: 100", font=("Helvetica", 14), bg="dark green", fg="white")
        self.score_label.grid(row=2, column=0, sticky="w", pady=5)

        # Bet entry
        self.bet_label = tk.Label(self.left_frame, text="Enter your bet:", font=("Helvetica", 12), bg="dark green", fg="white")
        self.bet_label.grid(row=3, column=0, sticky="w", pady=5)

        self.bet_entry = tk.Entry(self.left_frame, width=10)
        self.bet_entry.grid(row=4, column=0, sticky="w", pady=5)

        # Control buttons in the left frame
        button_style = {
            "font": ("Helvetica", 12),
            "width": 15,
            "bg": "#555555",
            "fg": "white",
            "activebackground": "#777777",
            "activeforeground": "white",
            "relief": "raised",
            "borderwidth": 3
        }

        self.bet_button = tk.Button(self.left_frame, text="Place Bet", command=self.place_bet, **button_style)
        self.bet_button.grid(row=5, column=0, pady=5, sticky="w")

        self.hit_button = tk.Button(self.left_frame, text="Hit", command=self.hit, state=tk.DISABLED, **button_style)
        self.hit_button.grid(row=6, column=0, pady=5, sticky="w")

        self.stand_button = tk.Button(self.left_frame, text="Stand", command=self.stand, state=tk.DISABLED, **button_style)
        self.stand_button.grid(row=7, column=0, pady=5, sticky="w")

        self.restart_button = tk.Button(self.left_frame, text="Restart", command=self.restart_game, **button_style)
        self.restart_button.grid(row=8, column=0, pady=5, sticky="w")

        self.next_round_button = tk.Button(self.left_frame, text="Next Round", command=self.next_round, state=tk.DISABLED, **button_style)
        self.next_round_button.grid(row=9, column=0, pady=5, sticky="w")


        # Canvas frame (only shows in maximized mode)
        self.canvas_frame = tk.Frame(self.right_frame, bg="dark green")

        self.player_canvas = tk.Canvas(self.canvas_frame, width=800, height=300, bg="green")
        self.player_canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=5, pady=5)

        self.dealer_canvas = tk.Canvas(self.canvas_frame, width=800, height=300, bg="green")
        self.dealer_canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True, padx=5, pady=5)

        # Back of card image
        self.back_of_card_image = ImageTk.PhotoImage(Image.open("images/Back_of_Card.png").resize((80, 120)))

        # Bind window resize events
        self.root.bind('<Configure>', self.check_window_state)

    def check_window_state(self, event=None):
        """Ensure canvas only shows in maximized mode."""
        if self.root.state() == 'zoomed':  # Only show canvas in maximized mode
            self.canvas_frame.pack(fill=tk.BOTH, expand=True)
        else:
            self.canvas_frame.pack_forget()  # Hide when restored down

    def show_rules(self):
        """Display a single-instance pop-up window with Blackjack rules."""
        if self.rules_window and self.rules_window.winfo_exists():
            # If window already exists, bring it to the front
            self.rules_window.lift()
            return

        self.rules_window = tk.Toplevel(self.root)
        self.rules_window.title("Blackjack Rules")
        self.rules_window.geometry("500x400")
        self.rules_window.configure(bg="white")

        # Scrollable Text Area
        text_frame = tk.Frame(self.rules_window, bg="white", padx=10, pady=10)
        text_frame.pack(fill=tk.BOTH, expand=True)

        rules_text = """\
Blackjack Rules

Objective:
- Beat the dealer by having a hand value closer to 21 without exceeding it.

Card Values:
- Cards 2 to 10 are worth their face value.
- Jacks, Queens, and Kings are worth 10 points each.
- Aces are worth either 1 or 11 points, depending on what is more beneficial for the hand.

Game Flow:
1. Place your bet.
2. Both you and the dealer receive two cards.
   - One of the dealer's cards is hidden.
3. Choose your action:
   - Hit: Take another card to improve your total hand value.
   - Stand: Keep your current hand as it is.
4. Dealer's Turn:
   - The dealer must hit if their hand is below 17.
   - The dealer must stand if their hand is 17 or higher.

Winning Conditions:
- You win if your hand is higher than the dealer's without exceeding 21.
- A hand consisting of an Ace and a 10-point card (a Blackjack) is an automatic win.
- If your total exceeds 21, you lose (this is called a bust).
- If you and the dealer have the same total, the round is a tie (push).

💰 Good luck and enjoy the game! 🎲
"""

        # Scrollable text widget
        text_widget = tk.Text(text_frame, wrap="word", font=("Helvetica", 12), bg="white", padx=5, pady=5)
        text_widget.insert(tk.END, rules_text)
        text_widget.config(state=tk.DISABLED)  # Make it read-only
        text_widget.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Add scrollbar
        scrollbar = tk.Scrollbar(text_frame, command=text_widget.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        text_widget.config(yscrollcommand=scrollbar.set)

    def place_bet(self):
        try:
            bet = int(self.bet_entry.get())
            if bet <= 0 or bet > self.chips.total:
                raise ValueError("Invalid bet amount.")
            self.chips.bet = bet
            self.score_label.config(text=f"Chips: {self.chips.total}")
            self.start_game()
        except ValueError as e:
            messagebox.showerror("Invalid Bet", str(e) or "Please enter a valid bet amount within your chip total.")

    def start_game(self):
        self.player_hand = Hand()
        self.dealer_hand = Hand()
        self.deck = Deck()
        self.deck.shuffle()
        for _ in range(2):
            self.player_hand.add_card(self.deck.deal_one())
            self.dealer_hand.add_card(self.deck.deal_one())
        self.update_labels()
        self.hit_button.config(state=tk.NORMAL)
        self.stand_button.config(state=tk.NORMAL)
        self.bet_button.config(state=tk.DISABLED)

    def display_cards(self, canvas, cards, reveal_all=True):
        canvas.delete("all")
        for idx, card in enumerate(cards):
            x = 50 + idx * 120
            y = 100
            if reveal_all or idx != 0:  # Hide the first card for the dealer if not revealed
                canvas.create_image(x, y, image=card.image, anchor=tk.CENTER)
            else:
                canvas.create_image(x, y, image=self.back_of_card_image, anchor=tk.CENTER)

    def update_labels(self, reveal_dealer=False):
        self.display_cards(self.player_canvas, self.player_hand.cards, reveal_all=True)
        self.display_cards(self.dealer_canvas, self.dealer_hand.cards, reveal_all=reveal_dealer)

        self.player_label.config(text=f"Player's Hand: {self.player_hand} (Total Value: {self.player_hand.value})")
        if reveal_dealer:
            self.dealer_label.config(text=f"Dealer's Hand: {self.dealer_hand} (Total Value: {self.dealer_hand.value})")
        else:
            dealer_cards = ["Hidden"] + [str(self.dealer_hand.cards[1]) if len(self.dealer_hand.cards) > 1 else "Hidden"]
            self.dealer_label.config(text=f"Dealer's Hand: {', '.join(dealer_cards)}")

    def hit(self):
        self.player_hand.add_card(self.deck.deal_one())
        self.update_labels()
        if self.player_hand.value > 21:
            self.end_round(player_bust=True)

    def stand(self):
        self.player_turn = False
        while self.dealer_hand.value < 17:
            self.dealer_hand.add_card(self.deck.deal_one())
        self.end_round()

    def end_round(self, player_bust=False):
        self.hit_button.config(state=tk.DISABLED)
        self.stand_button.config(state=tk.DISABLED)
        self.update_labels(reveal_dealer=True)
        if player_bust:
            messagebox.showinfo("Bust!", "You busted! Dealer wins.")
            self.chips.lose_bet()
        elif self.dealer_hand.value > 21 or self.player_hand.value > self.dealer_hand.value:
            messagebox.showinfo("Win!", "You win!")
            self.chips.win_bet()
        elif self.player_hand.value < self.dealer_hand.value:
            messagebox.showinfo("Lose!", "Dealer wins.")
            self.chips.lose_bet()
        else:
            messagebox.showinfo("Push", "It's a tie!")
        self.score_label.config(text=f"Chips: {self.chips.total}")
        if self.chips.total <= 0:
            messagebox.showinfo("Game Over", "You are out of chips! Restart to play again.")
            self.restart_game()
        else:
            self.next_round_button.config(state=tk.NORMAL)

    def restart_game(self):
        self.player_hand = Hand()
        self.dealer_hand = Hand()
        self.chips = Chips()
        self.update_labels()
        self.score_label.config(text=f"Chips: {self.chips.total}")
        self.bet_button.config(state=tk.NORMAL)
        self.hit_button.config(state=tk.DISABLED)
        self.stand_button.config(state=tk.DISABLED)
        self.next_round_button.config(state=tk.DISABLED)
        self.player_label.config(text="Player's Hand: []")
        self.dealer_label.config(text="Dealer's Hand: []")
        self.bet_entry.delete(0, tk.END)

    def next_round(self):
        self.player_hand = Hand()
        self.dealer_hand = Hand()
        self.deck = Deck()
        self.deck.shuffle()
        self.update_labels()
        self.bet_button.config(state=tk.NORMAL)
        self.hit_button.config(state=tk.DISABLED)
        self.stand_button.config(state=tk.DISABLED)
        self.next_round_button.config(state=tk.DISABLED)
        self.bet_entry.delete(0, tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    game = BlackjackGame(root)
    root.mainloop()